In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.layers import LSTM, Dense, Dropout, MultiHeadAttention, LayerNormalization

In [ ]:
! pip install mplfinance
! pip install pandas_ta

In [ ]:
data = pd.read_csv('/kaggle/input/eurusd/EURUSD-30-DS.csv', header=None, names=['Date','Open', 'High', 'Low', 'Close','Volume'])
data.head()

In [ ]:
import pandas_ta as ta
data['RSI_14'] = ta.rsi(data['Close'], length=14)
data['MA20'] = ta.ema(data['Close'], length=20)
data['MACD'] = ta.macd(data['Close'])['MACD_12_26_9']
data['ATR_4'] = ta.atr(data['High'], data['Low'], data['Close'], length=4)
data['ATR_14'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
data['ATR_40'] = ta.atr(data['High'], data['Low'], data['Close'], length=40)
bb = ta.bbands(data['Close'], length=20)
data['BB_upper'] = bb['BBU_20_2.0']
data['BB_lower'] = bb['BBL_20_2.0']
data['Stoch_K'] = ta.stoch(data['High'], data['Low'], data['Close'])['STOCHk_14_3_3']
data['CCI_20'] = ta.cci(data['High'], data['Low'], data['Close'], length=20)
data['adx_14'] = data.ta.adx(length=14)['ADX_14']
data['Candle_Range'] = data['High'] - data['Low']
data['Candle_Body'] = data['Close'] - data['Open']


# # اضافه کردن لگ‌های زمانی برای Average
# for lag in range(1, 6):  # لگ‌های 1 تا 5
#     data[f'Average_Lag_{lag}'] = data['Average'].shift(lag)

data['Rolling_Mean_10'] = data['Close'].rolling(window=10).mean()
data['Rolling_Std_20'] = data['Close'].rolling(window=20).std()
data['Rolling_Max_20'] = data['High'].rolling(window=20).max()
data['Rolling_Min_20'] = data['Low'].rolling(window=20).min()
data['Price_Change'] = (data['Close'] - data['Open']) / data['Open']
data['Stoch_D'] = ta.stoch(data['High'], data['Low'], data['Close'])['STOCHd_14_3_3']
data = data.dropna()

data['Date'] = pd.to_datetime(data['Date'], format='%d.%m.%Y %H:%M:%S.%f')
data['Hour_of_Day'] = data['Date'].dt.hour
data['Day_of_Week'] = data['Date'].dt.dayofweek

data['Average'] = ((data['Open'] + data['Close'] + data['High'] + data['Low']) / 4)

data = data.dropna()

In [ ]:
import seaborn as sns
correlation_matrix = data[['Open', 'High', 'Low', 'Close', 'Volume', 'RSI_14', 'MA20',
       'MACD', 'ATR_4', 'ATR_14', 'ATR_40', 'BB_upper', 'BB_lower', 'Stoch_K',
       'CCI_20', 'adx_14', 'Candle_Range', 'Candle_Body', 'Rolling_Mean_10',
       'Rolling_Std_20', 'Rolling_Max_20', 'Rolling_Min_20', 'Price_Change',
       'Stoch_D', 'Hour_of_Day', 'Day_of_Week', 'Average']].corr()
plt.figure(figsize=(25, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()

In [ ]:
def create_sequences(inputs, outputs, seq_length):
    X, y = [], []
    for i in range(len(inputs) - seq_length):
        X.append(inputs[i:i + seq_length])
        y.append(outputs[i + seq_length])  # 4 مقدار OHLC تایم‌فریم بعدی
    return np.array(X), np.array(y)

seq_length = 20
X, y = create_sequences(scaled_inputs, scaled_outputs, seq_length)

In [ ]:
train_size = int(len(X) * 0.8)
val_size = int(len(X) * 0.1)
X_train, X_val, X_test = X[:train_size], X[train_size:train_size + val_size], X[train_size + val_size:]
y_train, y_val, y_test = y[:train_size], y[train_size:train_size + val_size], y[train_size + val_size:]

print("تعداد نمونه‌های Train:", len(X_train))
print("تعداد نمونه‌های Validation:", len(X_val))
print("تعداد نمونه‌های Test:", len(X_test))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
import numpy as np

inputs = Input(shape=(seq_length, len(num_features))

# لایه‌های CNN
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# لایه‌های LSTM
x = LSTM(64, return_sequences=True)(x)
x = Dropout(0.4)(x)
x = BatchNormalization()(x)
x = LSTM(32, return_sequences=False)(x)
x = Dropout(0.4)(x)

# لایه‌های خروجی
x = Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.05))(x)
x = Dropout(0.3)(x)
outputs = Dense(1)(x)

# تعریف مدل
model = Model(inputs=inputs, outputs=outputs)

# بهینه‌ساز و کامپایل
optimizer = Adam(learning_rate=0.001)  # نرخ یادگیری اولیه بالاتر
model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])

# زمان‌بندی نرخ یادگیری
def lr_scheduler(epoch, lr):
    if epoch < 5:
        return float(lr)
    else:
        return float(lr * np.exp(-0.05))

lr_scheduler_callback = LearningRateScheduler(lr_scheduler)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# خلاصه مدل
model.summary()

In [ ]:
# آموزش مدل
history = model.fit(
    X_train, y_train,
    epochs=70,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, lr_scheduler_callback],
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss in training')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('MAE in training')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
predictions = model.predict(X_test)

# 10. برگردوندن به مقیاس اصلی
predictions_average = scalers_outputs['Average'].inverse_transform(predictions[:, 0].reshape(-1, 1))


y_test_average = scalers_outputs['Average'].inverse_transform(y_test[:, 0].reshape(-1, 1))

# 11. محاسبه معیارها برای هر مقدار
def calculate_metrics(y_true, y_pred, name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\nمعیارهای ارزیابی برای {name}:")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R² Score: {r2}")

calculate_metrics(y_test_average, predictions_average, "Average")



# 13. رسم پیش‌بینی‌ها
plt.figure(figsize=(20, 6))
plt.subplot(1, 1, 1)
plt.plot(y_test_average, label='real', color='blue')
plt.plot(predictions_average, label='predicted', color='red',linestyle='--')
plt.title('Average: actual vs predicted')
plt.legend()



plt.tight_layout()
plt.show()

In [ ]:
total_samples = 1000
n_samples_per_plot = 100  # تعداد نمونه‌ها در هر نمودار
n_plots = total_samples // n_samples_per_plot  # تعداد نمودارها (10)

# تنظیمات شکل کلی
plt.figure(figsize=(12, 8 * n_plots))  # ارتفاع شکل رو متناسب با تعداد نمودارها تنظیم می‌کنیم

# حلقه برای رسم 10 نمودار
for i in range(n_plots):
    # محاسبه بازه نمونه‌ها برای هر نمودار
    start_idx = i * n_samples_per_plot
    end_idx = (i + 1) * n_samples_per_plot
    
    # انتخاب داده‌ها برای این بازه
    y_test_subset = y_test_average[start_idx:end_idx]
    predictions_subset = predictions_average[start_idx:end_idx]
    
    # ایجاد زیرنمودار
    plt.subplot(n_plots, 1, i + 1)  # n_plots ردیف، 1 ستون، اندیس i+1
    
    # رسم خطوط
    plt.plot(y_test_subset, label='real', color='blue')
    plt.plot(predictions_subset, label='predicted', color='red')
    
    # اضافه کردن عنوان و لیبل‌ها
    plt.title(f'Average: actual vs predicted (Samples {start_idx} to {end_idx})')
    plt.ylabel('Average')
    plt.legend()
    
    # تنظیم محور x برای نمایش اندیس نمونه‌ها
    plt.xticks(np.arange(0, n_samples_per_plot, 10))  # تیک‌ها هر 10 نمونه

# تنظیمات نهایی و نمایش نمودار
plt.tight_layout()
plt.show()

In [ ]:
import mplfinance as mpf
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# فرض می‌کنیم data دیتافریم اولیه شماست که ستون‌های Date, Open, High, Low, Close, Average داره
# تبدیل ستون Date به فرمت datetime
data['Date'] = pd.to_datetime(data['Date'], format='mixed', dayfirst=True)

# تعداد نمونه‌های تست رو از X_test می‌گیریم
n_test_samples = len(X_test)  # تعداد نمونه‌های تست: 21186

# انتخاب داده‌های تست از دیتافریم data
test_data = data.iloc[-n_test_samples:].copy()
test_data.set_index('Date', inplace=True)
test_data.index = pd.DatetimeIndex(test_data.index)

# محدود کردن تعداد نمونه‌ها برای رسم (100 نمونه اول)
n_samples_to_plot = 100
test_data = test_data.iloc[:n_samples_to_plot]  # 100 نمونه اول

# فرض می‌کنیم predictions خروجی مدل شماست و برای داده‌های تست (X_test) هست
# predictions باید ابعاد (n_test_samples, 1) داشته باشه
predicted_average = predictions.flatten()  # تبدیل به آرایه یک‌بعدی
predicted_average = predicted_average[:n_samples_to_plot]  # محدود کردن به 100 نمونه اول

# اگه predictions نرمال‌سازی‌شده هست، باید به مقیاس اصلی برگردونیم
# فرض می‌کنیم scalers_outputs['Average'] همون اسکیلری هست که برای نرمال‌سازی Average استفاده کردی
predicted_average = scalers_outputs['Average'].inverse_transform(predicted_average.reshape(-1, 1)).flatten()

# ایجاد یک DataFrame برای Average پیش‌بینی‌شده با همان ایندکس
predicted_df = pd.DataFrame({
    'Predicted_Average': predicted_average
}, index=test_data.index)

# برای مقایسه، Average واقعی رو هم از test_data می‌گیریم
actual_average = test_data['Average'].values
actual_average_df = pd.DataFrame({
    'Actual_Average': actual_average
}, index=test_data.index)

# محاسبه معیارهای ارزیابی
mae = mean_absolute_error(actual_average, predicted_average)
rmse = np.sqrt(mean_squared_error(actual_average, predicted_average))
print(f"MAE on Test Data (First 100 Samples): {mae}")
print(f"RMSE on Test Data (First 100 Samples): {rmse}")

# تنظیمات برای رسم
fig = mpf.figure(figsize=(15, 10))
ax = fig.add_subplot(1, 1, 1)

# رسم کندل‌استیک‌ها
mpf.plot(test_data, type='candle', style='charles', ax=ax, volume=False)

# اضافه کردن خط Actual Average
ap1 = mpf.make_addplot(actual_average_df['Actual_Average'], type='line', color='red', linestyle='--', ax=ax, label='Actual Average')

# اضافه کردن خط Predicted Average
ap2 = mpf.make_addplot(predicted_df['Predicted_Average'], type='line', color='blue', linestyle='-',  ax=ax, label='Predicted Average')

# رسم کندل‌ها با هر دو خط
mpf.plot(test_data, type='candle', style='charles', ax=ax, volume=False, addplot=[ap1, ap2])

# اضافه کردن عنوان و لیبل‌ها
ax.set_title('Actual Candles with Actual vs Predicted Average (First 100 Samples of Test Data)')
ax.set_ylabel('Price')

# نمایش لجند
ax.legend()

# تنظیمات نهایی و نمایش نمودار
fig.tight_layout()
mpf.show()